You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [11]:
import pandas as pd

df1 = pd.read_csv("./datasets/office_addresses.csv")
df2 = pd.read_excel("./datasets/employee_information.xlsx")
df3 = pd.read_excel("./datasets/employee_information.xlsx", sheet_name = "emergency_contacts", header = None)
df3.columns = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']
df4 = pd.read_json("./datasets/employee_roles.json").T.reset_index().rename(columns = {'index': 'employee_id'})

In [ ]:
employees = pd.merge(df2,df3, on = 'employee_id', how = 'inner')
employees.drop(columns = ['last_name', 'first_name'], inplace = True)
employees = pd.merge(employees, df4, on = 'employee_id', how = 'inner')
employees

In [ ]:
employees_final = pd.merge(employees, df1, left_on = 'employee_country', right_on = 'office_country', how = 'outer')

for col_name in employees_final.columns:
    if 'office' in col_name:
        employees_final[col_name] = employees_final[col_name].fillna('Remote')

col_lists = employees_final.columns
new_col_lists = ['employee_id', 'employee_first_name', 'employee_last_name', 'title', 'team', 'monthly_salary',
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number', 'emergency_contact',
       'emergency_contact_number', 'relationship', 
        'office', 'office_country', 'office_city', 'office_street',
       'office_street_number']
employees_final = employees_final[new_col_lists]
employees_final